# Module 2 Final Project Submission
* Team Name: (Mia Iseman and Pat Fuller)  
* Repository URL:
* Presentation URL: 

# Goal: 
## Generate meaningful analytical insights to Universal Music Group based on their ratings and reviews. 


## Process - will this be hypothesis testing entirely? 
1. Understand the business needs
2. Understand the data
3. Prepare the data
4. Model
5. Evaluate
6. Deploy

## Business Understanding 

### Project Context 


### Business Needs


## Data Understanding 

## Data Preparation

## Modeling

## Evaluation

## Deployment 
(be sure to frame our limitations and what no-relationship means) 
